<a href="https://colab.research.google.com/github/MariaZharova/test_rec_systems/blob/main/Our_microsoft_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 

from collections import defaultdict
import category_encoders as ce
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

# Evaluation
from recommenders.evaluation.python_evaluation import precision_at_k

In [ ]:
! pip install scrapbook category_encoders recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 9.0 MB/s 
     |████████████████████████████████| 335 kB 66.1 MB/s 
     |████████████████████████████████| 355 kB 65.5 MB/s 
     |████████████████████████████████| 310 kB 72.0 MB/s 
     |████████████████████████████████| 197 kB 64.9 MB/s 
     |████████████████████████████████| 636 kB 58.4 MB/s 
     |████████████████████████████████| 61 kB 10.2 MB/s 
     |████████████████████████████████| 11.8 MB 58.2 MB/s 
     |████████████████████████████████| 4.4 MB 52.9 MB/s 
     |████████████████████████████████| 12.4 MB 32.4 MB/s 
     |████████████████████████████████| 387 kB 73.7 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 6.6 MB 68.1 MB/s 
     |████████████████████████████████| 132 kB 64.6 MB/s 
     |████████████████████████████████| 106 kB 71.9 MB/s 
     |██████████████████████████

In [2]:
my_data = pd.read_csv('internship_clickstream_data.csv')
print(my_data.shape) # (7458216, 8)
#my_data = my_data.iloc[:20000]
print(my_data.shape)
my_data.head()

(3216049, 8)
(3216049, 8)


,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd
0,2022-06-29 01:04:03,4b45e714d01842a7,16650505,ios,OpenOfferScreen,SearchResultsList,274266785.0,2022-06-29
1,2022-06-29 01:06:10,e688e3349b35430f,92346837,android,OpenOfferScreen,MapScreen,270671363.0,2022-06-29
2,2022-06-29 01:08:48,97c52e7a2e574f44,0bf81f10-ee3a-4543-a9ee-2bd12b4e4ec6,android,OpenOfferScreen,Undefined,272968099.0,2022-06-29
3,2022-06-29 01:09:39,d52e99bc7f7f4db8,84081279,ios,OpenOfferScreen,SearchResultsList,268313499.0,2022-06-29
4,2022-06-29 01:12:50,d49bf3352f64401c,0bf81f10-ee3a-4543-a9ee-2bd12b4e4ec6,android,OpenOfferScreen,Undefined,255933042.0,2022-06-29


In [3]:
my_data.dropna(inplace=True)
my_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3144973 entries, 0 to 3216047
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   timestamp   object 
 1   hit_id      object 
 2   uid         object 
 3   platform    object 
 4   event_name  object 
 5   screen      object 
 6   offer_id    float64
 7   ptn_dadd    object 
dtypes: float64(1), object(7)
memory usage: 215.9+ MB


In [4]:
# encode, start with 1
offer_encoder = {off: ind for ind, off in enumerate(my_data['offer_id'].unique())}
my_data['offer_id_enc'] = my_data['offer_id'].map(offer_encoder) + 1
uid_encoder = {uid: ind for ind, uid in enumerate(my_data['uid'].unique())}
my_data['uid_enc'] = my_data['uid'].map(uid_encoder) + 1

# sort by user id and time iteraction
my_data['timestamp'] = pd.to_datetime(my_data['timestamp'])
my_data.sort_values(by=['uid_enc', 'timestamp'], inplace=True)
my_data.head(10)

,timestamp,hit_id,uid,platform,event_name,screen,offer_id,ptn_dadd,offer_id_enc,uid_enc
1542174,2022-06-28 04:44:41,2ddc8a7e74484809,16650505,ios,OpenOfferScreen,Undefined,275080365.0,2022-06-28,19300,1
1542181,2022-06-28 04:46:54,a5ebdc71519949e4,16650505,ios,OpenOfferScreen,MapScreen,271174333.0,2022-06-28,132461,1
115025,2022-06-28 04:48:14,895cff66bfdc4b58,16650505,ios,OpenOfferScreen,MapScreen,274706741.0,2022-06-28,69924,1
115039,2022-06-28 04:53:13,6fd105b988a84f00,16650505,ios,OpenOfferScreen,MapScreen,249093727.0,2022-06-28,6070,1
6509,2022-06-28 04:53:23,0c2b184e62f04368,16650505,ios,OpenOfferScreen,MapScreen,249093727.0,2022-06-28,6070,1
6513,2022-06-28 04:54:23,289db0a4dfc344bf,16650505,ios,OpenOfferScreen,MapScreen,274743850.0,2022-06-28,6074,1
6532,2022-06-28 04:59:12,21df94e71e4b4df7,16650505,ios,OpenOfferScreen,MapScreen,264617112.0,2022-06-28,6093,1
115065,2022-06-28 04:59:31,c41f10e93b64463d,16650505,ios,OpenOfferScreen,MapScreen,274477379.0,2022-06-28,87564,1
2639240,2022-06-28 05:06:33,c3fd8fbbbe194d71,16650505,ios,OpenOfferScreen,MapScreen,269315638.0,2022-06-28,303079,1
2639245,2022-06-28 05:08:19,e9bc63550f1b4606,16650505,ios,OpenOfferScreen,MapScreen,228293739.0,2022-06-28,28834,1


In [5]:
# create .txt file for input to model
my_data[['offer_id_enc',	'uid_enc']].to_csv('out.txt', sep="\t", header=False, index=False)

In [6]:
# create specia; data format for SAS
data = SASRecDataSet(filename='out.txt', col_sep='\t')
# split into train, test and validation
data.split()

In [7]:
# model variables
num_epochs = 5
batch_size = 128
RANDOM_SEED = 100  # Set None for non-deterministic result

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 2         # number of transformer blocks
hidden_units = 100     # number of units in the attention calculation
num_heads = 1          # number of attention heads
dropout_rate = 0.1     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = 100     # number of negative examples per positive example


In [8]:
# sample negative examples
sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [9]:
model = SASREC(item_num=data.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [100, 100],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test)

In [10]:
with Timer() as train_time:
    t_test = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)


epoch: 5, test (NDCG@10: 0.3843548958767651, HR@10: 0.4900697799248524)


In [11]:
t_test # встроенно оценивает ndcg@10 (Normalized discounted cumulative gain) и Hit@10

(0.3843548958767651, 0.4900697799248524)

In [12]:
print(data.usernum)
print(model.num_neg_test)

763017
100


In [13]:
import random
from tqdm import tqdm

def get_predictions(data):
    """
        Модифицированный метод evaluation класса SASREC,
        главный результат - получаем предсказания для всех пользователей из датасета
    """
    usernum = data.usernum # max № of user
    itemnum = data.itemnum # max № of item
    train = data.user_train
    valid = data.user_valid
    test = data.user_test

    pred_dict = {}
    all_inputs = {}
    
    # насэмплим рандомных 10000 пользователей (или меньше, если их разнообразие небольшое:))
    if usernum > 10000:
        users = random.sample(range(1, usernum + 1), 10000)
    else:
        users = range(1, usernum + 1)
    
    # для каждого пользователя делаем оценку
    for u in tqdm(users, ncols=70, leave=False, unit="b"):

        if len(train[u]) < 1 or len(test[u]) < 1: # если для текущего пользователя нет ничего в train или test => continue
            continue
        # для input_seq
        seq = np.zeros([model.seq_max_len], dtype=np.int32)
        idx = model.seq_max_len - 1
        seq[idx] = valid[u][0]
        idx -= 1
        for i in reversed(train[u]): 
            seq[idx] = i
            idx -= 1
            if idx == -1: # если нет в train и valid => break
                break
        # для candidate
        rated = set(train[u]) # то, что оценил пользователь, из train'a
        rated.add(0)
        item_idx = [test[u][0]] # первым в последовательность помещаем тык из теста
        for _ in range(model.num_neg_test): # размер последовательностей получается фиксированный, задаётся в параметрах модели при инициализации
                                            # https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/sasrec_amazon.ipynb
            t = np.random.randint(1, itemnum + 1)
            while t in rated:
                t = np.random.randint(1, itemnum + 1) # генерим рандомно (?)
            item_idx.append(t)

        inputs = {}
        inputs["user"] = np.expand_dims(np.array([u]), axis=-1) # просто номер пользователя
        inputs["input_seq"] = np.array([seq]) # входная последовательность что тыкнул пользователь - ИЗ TRAIN И VALID!
        inputs["candidate"] = np.array([item_idx]) # объявки, для которых будем вычислять логиты
        all_inputs[inputs["user"][0][0]] = inputs # словарик для всех inputs

        # добавляем каждого пользователя в словарик предиктов
        pred_dict[inputs["user"][0][0]] = model.predict(inputs)

    return pred_dict, all_inputs

In [14]:
pred_dict, inputs = get_predictions(data)

In [17]:
# посмотрим, что input_seq - это то, что нажал пользователь из train и valid
display(inputs[197834]) # user_id для примера
print("\n")
print("Из train и valid:", data.user_train[197834], data.user_valid[197834])
print("Из test:", data.user_test[197834])

{'candidate': array([[307241, 248461, 229117, 260154, 237033, 335129, 303309, 172450,
         195503, 110601, 299518, 304479, 156651,  54203,  95699, 314017,
         248911, 215058, 140867, 347412, 168500, 113155, 269288, 342205,
         158123, 250891,  75254, 120293, 184029, 303561, 136252, 169396,
         102537, 214324, 342691, 163986, 203657, 316186,  18791,  67661,
         101189, 306793, 299807,  82952, 213061, 237159, 208950, 284473,
         165057,   6877,  29577, 290357, 149652, 289100, 192830, 118124,
          87978, 106419, 267555, 234797, 333476, 208098,   7350, 345894,
         190867, 152449, 183609, 127762,  60206, 310756,  67167,  62922,
         174200, 187379,  81389,  13669,  35986, 198208, 221138, 194622,
         144272,  46629, 209936,  79067,   5143, 225702,  89215,  85640,
         177801,  75797, 329840, 279488, 104058, 239999, 324915, 284918,
         163612,  76766, 262816, 272067, 265842]]),
 'input_seq': array([[     0,      0,      0,      0,      



Из train и valid: [1241, 34698, 113040] [269082]
Из test: [307241]


In [18]:
print(len(pred_dict)) # осталось столько пользователей (для которых было что-то в train && test)
print(inputs[197834]['candidate'].shape) # всего 100 штук, сколько и заказывали в num_neg_test при создании модели

3871
(1, 101)


In [20]:
# посмотреть на пример что в pred_dict
pred_dict[197834]

<tf.Tensor: shape=(1, 101), dtype=float32, numpy=
array([[-13.250863  , -13.005783  ,  -7.488702  ,  -6.4093566 ,
         -9.790439  ,  -8.110545  , -10.98637   ,  -7.1202674 ,
         -3.9585311 ,  -5.7984076 , -10.5582905 , -14.404372  ,
        -10.619322  , -10.923225  ,  -0.83093035, -12.556803  ,
         -3.7429867 , -12.506489  ,  -7.5008426 ,  -8.876704  ,
        -11.462374  ,  -1.921906  ,  -9.318563  , -13.772251  ,
         -7.393725  ,  -2.1304467 ,  -5.857313  ,  -4.3034496 ,
         -5.3921824 ,  -4.5420237 ,  -6.739258  ,   3.4749246 ,
        -11.309206  ,  -8.276506  , -15.021044  , -11.154677  ,
         -8.818069  ,  -8.682829  , -10.000908  , -13.711383  ,
         -7.4390125 ,  -8.544452  , -11.319796  ,  -6.9906173 ,
         -3.4864435 ,  -5.3349714 , -10.110867  ,  -9.941314  ,
         -3.5174072 ,  -5.722214  ,   1.6308593 , -10.838854  ,
         -7.312593  ,  -7.5536356 ,  -2.197273  ,  -4.6795154 ,
         -8.091921  ,  -3.779712  ,  -5.758985  , -14.

In [21]:
# нужно связать значения логитов и кандидатов, выбрать самые вероятные значения и проверить, были ли они в тесте
# сразу расчёт precision@k
k = 5
fin_prec = 0
for key in pred_dict.keys():
  tmp = np.stack((pred_dict[key][0], inputs[key]['candidate'][0]), axis=-1)
  tmp = tmp[tmp[:, 0].argsort()] # сортировка по вероятностям (логитам)
  topk = tmp[-k:, 1].astype(int)
  # проверим, есть ли этот топk в тесте
  tmp_prec = 0
  for val in topk:
    if val in data.user_test[key]:
      tmp_prec += 1/k
  fin_prec += tmp_prec

fin_prec /= len(pred_dict.keys())
print("ОТВЕТ", fin_prec)

ОТВЕТ 0.08840092999224726


In [25]:
np.array(pred_dict[197834][0])#.shape

array([-13.250863  , -13.005783  ,  -7.488702  ,  -6.4093566 ,
        -9.790439  ,  -8.110545  , -10.98637   ,  -7.1202674 ,
        -3.9585311 ,  -5.7984076 , -10.5582905 , -14.404372  ,
       -10.619322  , -10.923225  ,  -0.83093035, -12.556803  ,
        -3.7429867 , -12.506489  ,  -7.5008426 ,  -8.876704  ,
       -11.462374  ,  -1.921906  ,  -9.318563  , -13.772251  ,
        -7.393725  ,  -2.1304467 ,  -5.857313  ,  -4.3034496 ,
        -5.3921824 ,  -4.5420237 ,  -6.739258  ,   3.4749246 ,
       -11.309206  ,  -8.276506  , -15.021044  , -11.154677  ,
        -8.818069  ,  -8.682829  , -10.000908  , -13.711383  ,
        -7.4390125 ,  -8.544452  , -11.319796  ,  -6.9906173 ,
        -3.4864435 ,  -5.3349714 , -10.110867  ,  -9.941314  ,
        -3.5174072 ,  -5.722214  ,   1.6308593 , -10.838854  ,
        -7.312593  ,  -7.5536356 ,  -2.197273  ,  -4.6795154 ,
        -8.091921  ,  -3.779712  ,  -5.758985  , -14.611725  ,
       -14.645371  ,  -9.082115  ,  -5.326075  ,  -5.53

In [23]:
inputs[197834]['candidate'][0]#.shape

array([307241, 248461, 229117, 260154, 237033, 335129, 303309, 172450,
       195503, 110601, 299518, 304479, 156651,  54203,  95699, 314017,
       248911, 215058, 140867, 347412, 168500, 113155, 269288, 342205,
       158123, 250891,  75254, 120293, 184029, 303561, 136252, 169396,
       102537, 214324, 342691, 163986, 203657, 316186,  18791,  67661,
       101189, 306793, 299807,  82952, 213061, 237159, 208950, 284473,
       165057,   6877,  29577, 290357, 149652, 289100, 192830, 118124,
        87978, 106419, 267555, 234797, 333476, 208098,   7350, 345894,
       190867, 152449, 183609, 127762,  60206, 310756,  67167,  62922,
       174200, 187379,  81389,  13669,  35986, 198208, 221138, 194622,
       144272,  46629, 209936,  79067,   5143, 225702,  89215,  85640,
       177801,  75797, 329840, 279488, 104058, 239999, 324915, 284918,
       163612,  76766, 262816, 272067, 265842])

In [24]:
data.user_test[197834]

[307241]